In [1]:
# -*- coding: utf-8 -*-
"""XGBoostWB_Traffic
"""

import sys
#Import Libraries
#import tensorflow as tf
import pandas as pd
import numpy as np
import os
#import matplotlib
#import matplotlib.pyplot as plt
import random
# %matplotlib inline
import shutil
from random import shuffle
random.seed(42) 
import itertools

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from datetime import datetime, timedelta
from sklearn import preprocessing

%load_ext autoreload
%autoreload 2
from GBRT_for_TSF.utils import *


In [2]:
include_features = True
include_motif_information = True

num_periods_output = 24 #to predict
num_periods_input=24 #input

ALL_Test_Data=[]
ALL_Test_Prediction=[]

file_name = 'traffic.npy'
data_path=r'../data/' + file_name
data=np.load(data_path)
data= data[0:90,:]
# data= data[0:2,:]
data=pd.DataFrame(data)

In [3]:

"""## preprocessing"""

def New_preprocessing(TimeSeries):
   Data=[]
   start_date=datetime(2012, 1, 1,00,00,00) # define start date
   for i in range(0,len(TimeSeries)):
      record=[]
      record.append(TimeSeries[i])
      record.append(start_date.month)
      record.append(start_date.day)
      record.append(start_date.hour)
      record.append(start_date.weekday())
      record.append(start_date.timetuple().tm_yday)
      record.append(start_date.isocalendar()[1])
      #print(start_date.month,' ',start_date.day,' ',start_date.hour,' ',start_date.weekday(),' ',start_date.timetuple().tm_yday,' ',start_date.isocalendar()[1])
      start_date=start_date+ timedelta(hours=1)
      #print('year',start_date.year,'Month:',start_date.month,' day:',start_date.day,' hour:',start_date.hour)
      Data.append(record)
   ########## change list of lists to df ################
   headers=['traffic','month','day','hour','day_of_week','day_of_year','week_of_year']
   Data_df = pd.DataFrame(Data, columns=headers)
   sub=Data_df.iloc[:,1:]
   #Normalize features to be from -0.5 to 0.5 as mentioned in the paper
   New_sub= preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
   Normalized_Data_df=pd.DataFrame(np.column_stack([Data_df.iloc[:,0],New_sub]), columns=headers)
   
   #################################################################################################
   # cut training and testing training is 10392
   Train=Normalized_Data_df.iloc[0:10392,:]
   Train=Train.values
   Train = Train.astype('float32')
   Test=Normalized_Data_df.iloc[10392-num_periods_input:,:]
   Test=Test.values
   Test = Test.astype('float32')
   Number_Of_Features=7
   
   ############################################ Windowing ##################################
   end=len(Train)
   start=0
   next=0
   x_batches=[]
   y_batches=[]  
   count=0
   while next+(num_periods_input)<end:
        next=start+num_periods_input
        x_batches.append(Train[start:next,:])
        y_batches.append(Train[next:next+num_periods_output,0])
        start=start+1
   y_batches=np.asarray(y_batches)
   y_batches = y_batches.reshape(-1, num_periods_output, 1) 
   #print('Length of y batches :',len(y_batches),' ',num_periods_input,' ',num_periods_output)
   #print(x_batches)
   x_batches=np.asarray(x_batches) 
   x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)  
   
   ############################################ Windowing ##################################
   end_test=len(Test)
   start_test=0
   next_test=0
   x_testbatches=[]
   y_testbatches=[]
   while next_test+(num_periods_input)<end_test:
        next_test=start_test+num_periods_input
        x_testbatches.append(Test[start_test:next_test,:])
        y_testbatches.append(Test[next_test:next_test+num_periods_output,0])
        start_test=start_test+num_periods_input
   y_testbatches=np.asarray(y_testbatches)
   y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)   
   x_testbatches=np.asarray(x_testbatches)
   x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features) 

   if include_features:
       x_batches = x_batches[:, :, :]
       x_testbatches = x_testbatches[:, :, :]
   else:
       x_batches = x_batches[:, :, [0]]
       x_testbatches = x_testbatches[:, :, [0]]
       
   print('len x_batches',x_batches.shape,' len y_batches',y_batches.shape)
   print('len x_testbatches',x_testbatches.shape,' len y_testbatches',y_testbatches.shape)
   return x_batches, y_batches, x_testbatches, y_testbatches

In [4]:
x_batches_Full=[]
y_batches_Full=[]
X_Test_Full=[]
Y_Test_Full=[]
#len(data)
for i in range(0,len(data)):
    x_batches=[]
    y_batches=[]
    X_Test=[]
    Y_Test=[]
    TimeSeries=data.iloc[i,:]
    x_batches, y_batches,X_Test,Y_Test=New_preprocessing(TimeSeries)          
    for element1 in (x_batches):
        x_batches_Full.append(element1)
        
    for element2 in (y_batches):
        y_batches_Full.append(element2)
                    
    for element5 in (X_Test):
        X_Test_Full.append(element5)
        
    for element6 in (Y_Test):
        Y_Test_Full.append(element6)
    print('Finished processing time series', i+1, 'of', len(data))


len x_batches (10345, 24, 7)  len y_batches (10345, 24, 1)
len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)
Finished processing time series 1 of 90
len x_batches (10345, 24, 7)  len y_batches (10345, 24, 1)
len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)
Finished processing time series 2 of 90
len x_batches (10345, 24, 7)  len y_batches (10345, 24, 1)
len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)
Finished processing time series 3 of 90
len x_batches (10345, 24, 7)  len y_batches (10345, 24, 1)
len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)
Finished processing time series 4 of 90
len x_batches (10345, 24, 7)  len y_batches (10345, 24, 1)
len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)
Finished processing time series 5 of 90
len x_batches (10345, 24, 7)  len y_batches (10345, 24, 1)
len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)
Finished processing time series 6 of 90
len x_batches (10345, 24, 7)  len y_batches (10345, 

In [5]:
xgboost_parameters = {
    "learning_rate": 0.2,
    "n_estimators": 800,
    "max_depth": 8,
    "min_child_weight": 1,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 1,
    #  seed=42,silent=False
    "random_state": 42,
    "verbosity": 1
}

In [6]:
evaluate_with_xgboost(num_periods_output, x_batches_Full, y_batches_Full, X_Test_Full, Y_Test_Full, xgboost_parameters)

RMSE:  0.0370458
WAPE:  0.36180034
MAE:  0.019537142


# References
1. [GBRT-for-TSF/XGBoost_(W-b)/Univariate/xgboostWB_trafffic.py at main · Daniela-Shereen/GBRT-for-TSF](https://github.com/Daniela-Shereen/GBRT-for-TSF/blob/main/XGBoost_(W-b)/Univariate/xgboostWB_trafffic.py)

# Results
- data= data[0:90,:]
```
With Original Features
RMSE:  0.0370458
WAPE:  0.36180034
MAE:  0.019537142

Without Original Features
RMSE:  0.03838941
WAPE:  0.37899876
MAE:  0.020465853
```
- data= data[0:2,:]
```
With Original Features
RMSE:  0.03183206
WAPE:  0.32806885
MAE:  0.014425411

Without Original Features
RMSE:  0.03225562
WAPE:  0.3312612
MAE:  0.01456578
```

In [7]:
import datetime
print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")
###
###
###

This notebook was last run end-to-end on: 2025-08-18 00:15:08.002361

